In [1]:
import time
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
from torch.autograd import Variable

### Dataloader

In [2]:
class myDataset(Dataset):
    def __init__(self, csv_path, root_dir, status=1):
        self.root_dir = root_dir
        
        self.data_info = pd.read_csv(csv_path)
        #image path
        self.img_array = np.asarray(self.data_info.iloc[0:,1])
        #labels
        self.labels = np.asarray(self.data_info.iloc[0:,2], dtype=np.float32)
        #length of data
        self.data_len = len(self.data_info.index)
        #transformations on the batch
            #if status = 1 ===> transforms on train images
            #if status = 0 ===> transforms on val images
        if status == 1:
            self.transformations = transforms.Compose([transforms.Resize((256, 256)),
                                                       transforms.RandomRotation((90, 180)),
                                                       transforms.ToTensor()])
        elif status == 0:
            self.transformations = transforms.Compose([transforms.Resize((256, 256)),
                                                       transforms.ToTensor()])
        else:
            raise ValueError('Status should be 1 or 0')
    
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.img_array[idx] + '.jpeg')
        image = Image.open(img_path,'r')
        img_as_tensor = self.transformations(image)
        
        return {'image':img_as_tensor,'label':self.labels[idx]}
        #return sample
    

### Train

In [ ]:
def train_model(model, dataloader_train, dataloader_val, criterion, optimizer, scheduler, num_epochs):
    
    start_time = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_losses = []
    val_losses = []
    train_acc = []
    val_acc = []
    
    for epoch in range(num_epochs):
        print 'Epoch {}/{}'.format(epoch, num_epochs - 1)
        print '-'*10
        
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            num_correct = 0
            
            if phase == 'train':
                dataset = dataloader_train
            else:
                dataset = dataloader_val
                
            for idx, batch in enumerate(dataset):
                data = batch['image']
                data = Variable(data)
                labels = batch['label']
                labels = labels.long()
                labels = Variable(labels)
                if use_cuda:
                    data = data.cuda()
                    labels = labels.cuda()

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    output = model(data)
                    loss = criterion(output, labels)
                    _, preds = torch.max(output.data, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                running_loss += loss.item()*data.size(0)
                num_correct += torch.sum(preds == labels.data)
                del data, labels, output, preds
                torch.cuda.empty_cache()
                break
            epoch_loss =  running_loss / dataset.__len__()
            epoch_acc = num_correct.double() / dataset.__len__() 
            
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                val_losses.append(epoch_loss)
                val_acc.append(epoch_acc)
                
            print '{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            break    
        print 
        print '='*70
        print
        break
    time_elapsed = time.time() - start_time
    print 'Training complete in {:.0f}mins {:.0f}secs'.format(time_elapsed // 60, time_elapsed % 60)
    print 'Best accuracy {:.4f}'.format(best_acc * 100)
    
    model.load_state_dict(best_model_wts) #load it again to return it at the main function
    model.save_state_dict('model_name.pt')
    
    return model, train_losses, train_acc, val_losses, val_acc   

In [8]:
def train_model_cpu(model, dataloader_train, dataloader_val, criterion, optimizer, scheduler, num_epochs):
    
    start_time = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print 'Epoch {}/{}'.format(epoch, num_epochs - 1)
        print '-'*10
        
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            
            num_correct = 0
            
            if phase == 'train':
                dataset = dataloader_train
            else:
                dataset = dataloader_val
                
            for idx, batch in enumerate(dataset):
                data = batch['image']
                data = Variable(data)
                labels = batch['label']
                labels = labels.long()
                labels = Variable(labels)
                print'labels', labels

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    output = model(data)
                    loss = criterion(output, labels)
                    print'loss {}'.format(loss.item())
                    _, preds = torch.max(output.data, 1)
                    print'predictions', preds
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                running_loss += loss.item()*data.size(0)
                print'data size {}'.format(data.size(0))
                print'running loss {}'.format(running_loss)
                num_correct += torch.sum(preds == labels.data)
                print'number of correct hits {}'.format(num_correct)
                #del data, labels, output, preds
                #torch.cuda.empty_cache()
                #break
            epoch_loss =  running_loss / dataset.__len__()
            epoch_acc = num_correct.double() / dataset.__len__()    
           
            print '{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            #break    
        print 
        print '='*70
        print
        break
    time_elapsed = time.time() - start_time
    print 'Training complete in {:.0f}mins {:.0f}secs'.format(time_elapsed // 60, time_elapsed % 60)
    print 'Best accuracy {:.4f}'.format(best_acc)
    
    model.load_state_dict(best_model_wts)
    return model, best_acc   

### Prepare data

In [5]:
#csv_path = '/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/train/labels.csv'
root_dir = '/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/'
trainset = myDataset(os.path.join(root_dir,'train/labels.csv'), os.path.join(root_dir,'train'))
dataloader_train  = DataLoader(dataset=trainset,
                         batch_size=10,
                         shuffle=True,
                         num_workers=4,
                         pin_memory=True)
val_set = myDataset(os.path.join(root_dir,'val/labels.csv'), os.path.join(root_dir,'val'))
dataloader_val = DataLoader(dataset=val_set,
                         batch_size=10,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)
testset =  myDataset(os.path.join(root_dir,'test/labels.csv'), os.path.join(root_dir,'test'), status=0)
dataloader_test = DataLoader(dataset=testset,
                         batch_size=15,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)

### Model

In [6]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(64, 128, 3)
        self.norm1 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, 3)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv7 = nn.Conv2d(128, 256, 3)
        self.norm2 = nn.BatchNorm2d(256)
        self.conv8 = nn.Conv2d(256, 256, 3)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(256*12*12,4096)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(4096, 5)
    
    def forward(self, input):
        x = self.pool1(self.conv2(F.relu(self.conv1(input))))
        x = self.pool2(self.conv4(F.relu(self.conv3(x))))
        x = self.pool3(self.conv6(F.relu(self.norm1(self.conv5(x)))))
        x = self.pool4(self.conv8(F.relu(self.norm2(self.conv7(x)))))
        #flatten the tensor for the FC
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
        #return x
    

### Confusion matrix

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = 'd'
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    
def plot_conf_matrix(y_test, y_pred, class_names):
    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    print('CCR = {}'.format(np.trace(cnf_matrix) / len(y_test)))
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names, title='Confusion Matrix')
    plt.show()

### Test model

In [ ]:
def test_model(model, dataloader_test, criterion, optimizer, scheduler, num_epochs= 25):
    model.eval()
    num_correct = 0
    test_losses = []
    running_loss = 0.0
    accuracy = 0
    total = dataloader_test.__len__()
    for idx, batch in enumerate(dataloader_test):
        data = batch['image']
        data = Variable(data)
        labels = batch['label']
        labels = labels.long()
        labels = Variable(labels)
        if use_cuda:
            data = data.cuda()
            labels = labels.cuda()
        output = model(data)
        loss = criterion(output, labels)
        _,preds = torch.max(output.data, 1)
        running_loss += loss.data[0]*data.size(0)
        num_correct += torch.sum(preds == labels.data)
        true.extend(labels.data.cpu().numpy().tolist())
        pred.extend(preds.cpu().numpy().tolist())
        
    accuracy = num_correct/total
    return accuracy

### Main CPU 

In [9]:
#use_cuda = torch.cuda.is_available()
model = ConvNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.NLLLoss()
#criterion = nn.CrossEntropyLoss()
accuracy =0
num_epochs = 25
mymodel, accuracy=train_model_cpu(model, dataloader_train, dataloader_val, criterion, optimizer, scheduler, num_epochs=num_epochs)
#### plotting ####
#x = np.arange(num_epochs)


Epoch 0/24
----------
labels tensor([ 0,  0,  0,  3,  0,  0,  0,  0,  0,  0])
loss 1.54013991356
predictions tensor([ 0,  0,  4,  0,  1,  0,  0,  0,  3,  0])
data size 10
running loss 15.4013991356
number of correct hits 6
labels tensor([ 2,  2,  0,  0,  0,  1,  3,  0,  0,  1])
loss 2.76373910904
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 43.038790226
number of correct hits 11
labels tensor([ 0,  0,  0,  2,  0,  0,  0,  0,  2,  0])
loss 1.31264185905
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 56.1652088165
number of correct hits 19
labels tensor([ 1,  0,  2,  0,  0,  2,  0,  0,  2,  0])
loss 1.24166834354
predictions tensor([ 0,  0,  0,  0,  3,  0,  0,  0,  0,  0])
data size 10
running loss 68.581892252
number of correct hits 24
labels tensor([ 0,  0,  0,  0,  1,  0,  0,  1,  2,  0])
loss 1.40445446968
predictions tensor([ 2,  0,  2,  2,  2,  2,  0,  2,  0,  2])
data size 10
running loss 82.62643694

data size 10
running loss 446.790694296
number of correct hits 275
labels tensor([ 0,  0,  0,  1,  0,  0,  0,  0,  2,  0])
loss 1.43693804741
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 461.16007477
number of correct hits 283
labels tensor([ 0,  0,  1,  1,  2,  2,  0,  0,  0,  0])
loss 1.01371884346
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 471.297263205
number of correct hits 289
labels tensor([ 2,  0,  1,  0,  0,  0,  0,  0,  0,  0])
loss 0.858507990837
predictions tensor([ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 479.882343113
number of correct hits 298
labels tensor([ 1,  2,  1,  0,  0,  3,  0,  1,  0,  0])
loss 1.37051463127
predictions tensor([ 0,  0,  2,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 493.587489426
number of correct hits 303
labels tensor([ 0,  1,  0,  3,  3,  2,  0,  1,  0,  0])
loss 1.78411543369
predictions tensor([ 0,  0,  0,  0,  0,  0,  0

loss 0.401836693287
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 893.33917141
number of correct hits 559
labels tensor([ 2,  0,  1,  0,  0,  0,  0,  0,  0,  1])
loss 1.00962734222
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 903.435444832
number of correct hits 566
labels tensor([ 0,  0,  0,  0,  0,  3,  0,  0,  0,  4])
loss 0.985188305378
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 913.287327886
number of correct hits 574
labels tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  1,  2])
loss 0.828005969524
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 921.567387581
number of correct hits 582
labels tensor([ 2,  0,  0,  0,  0,  1,  2,  0,  0,  0])
loss 1.08166909218
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 932.384078503
number of correct hits 589
labels tensor([ 2,  3,  0,  2,  0,  2, 

data size 10
running loss 1324.20807987
number of correct hits 829
labels tensor([ 0,  0,  0,  0,  4,  0,  2,  0,  0,  0])
loss 0.884171307087
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 1333.04979295
number of correct hits 837
labels tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
loss 0.538504302502
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 1338.43483597
number of correct hits 847
labels tensor([ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0])
loss 0.554490864277
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 1343.97974461
number of correct hits 856
labels tensor([ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0])
loss 0.392922222614
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 1347.90896684
number of correct hits 865
labels tensor([ 1,  0,  0,  0,  0,  4,  0,  0,  2,  0])
loss 1.10533642769
predictions tensor([ 0,  0,  0,  0,  0,  0

loss 0.693443775177
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 1716.81137592
number of correct hits 1129
labels tensor([ 2,  2,  0,  0,  0,  0,  4,  2,  0,  2])
loss 1.38601565361
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 1730.67153245
number of correct hits 1134
labels tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  1,  0])
loss 0.709245383739
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 1737.76398629
number of correct hits 1143
labels tensor([ 0,  0,  0,  1,  1,  0,  0,  0,  0,  0])
loss 0.897833347321
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 1746.74231976
number of correct hits 1151
labels tensor([ 0,  0,  0,  1,  0,  0,  0,  0,  0,  0])
loss 0.676973223686
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 1753.512052
number of correct hits 1160
labels tensor([ 0,  1,  0,  0,  2,

data size 10
running loss 2064.93665338
number of correct hits 1436
labels tensor([ 2,  0,  0,  0,  0,  0,  0,  0,  2,  0])
loss 0.51802021265
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2070.1168555
number of correct hits 1444
labels tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  2,  0])
loss 0.481470108032
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2074.93155658
number of correct hits 1453
labels tensor([ 0,  0,  0,  0,  0,  2,  1,  0,  0,  0])
loss 0.690883994102
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2081.84039652
number of correct hits 1461
labels tensor([ 0,  0,  2,  1,  1,  2,  4,  1,  0,  0])
loss 2.49252390862
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2106.76563561
number of correct hits 1465
labels tensor([ 0,  0,  1,  0,  0,  0,  0,  0,  0,  0])
loss 0.510551571846
predictions tensor([ 0,  0,  0,  0,  0,

loss 0.33855432272
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2424.11583424
number of correct hits 1746
labels tensor([ 0,  0,  0,  0,  0,  3,  0,  0,  0,  0])
loss 0.444992870092
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2428.56576294
number of correct hits 1755
labels tensor([ 0,  1,  0,  2,  0,  0,  2,  1,  0,  2])
loss 1.85588991642
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2447.1246621
number of correct hits 1760
labels tensor([ 0,  0,  2,  0,  2,  0,  0,  0,  0,  0])
loss 0.645306944847
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2453.57773155
number of correct hits 1768
labels tensor([ 0,  0,  1,  0,  0,  1,  0,  0,  0,  0])
loss 0.738888859749
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2460.96662015
number of correct hits 1776
labels tensor([ 0,  0,  0,  0,  0,

data size 10
running loss 2858.7830697
number of correct hits 2010
labels tensor([ 0,  0,  0,  0,  0,  2,  1,  2,  3,  0])
loss 1.29037845135
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2871.68685421
number of correct hits 2016
labels tensor([ 0,  0,  0,  2,  2,  0,  0,  0,  0,  0])
loss 0.837587535381
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2880.06272957
number of correct hits 2024
labels tensor([ 0,  0,  0,  0,  2,  0,  0,  0,  0,  0])
loss 0.668532371521
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2886.74805328
number of correct hits 2033
labels tensor([ 0,  0,  0,  2,  0,  0,  0,  0,  0,  0])
loss 0.552187979221
predictions tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
data size 10
running loss 2892.26993307
number of correct hits 2042
labels tensor([ 0,  3,  1,  0,  0,  0,  2,  2,  2,  0])
loss 1.57606577873
predictions tensor([ 0,  0,  0,  0,  0,

Process Process-10:
Process Process-11:
Process Process-12:
Process Process-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/eslam/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/eslam/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/eslam/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/eslam/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
    self.run()
    self.run()
  File "/home/eslam/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/eslam/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/eslam/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/esl

KeyboardInterrupt: 

### Main

In [ ]:
use_cuda = torch.cuda.is_available()
model = ConvNet().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
#criterion = nn.NLLLoss().cuda()
criterion = nn.NLLLoss().cuda()
accuracy =0
num_epochs = 25
true = []
pred = []
class_names = ['0', '1', '2', '3', '4']
mymodel, train_losses, train_acc, val_losses, val_acc=train_model(model, dataloader_train, dataloader_val, criterion, optimizer, scheduler, num_epochs=25)
accuracy = test_model(mymodel, dataloader_test, criterion, optimizer, scheduler, num_epochs = 25)
plot_conf_matrix(true, pred, class_names)

### plotting ###
epochsRange = np.array(range(num_epochs))
plt.plot(epochsRange, train_losses, label='Training')
plt.plot(epochsRange, val_losses, label='Validation')
plt.title('Loss')
plt.legend(loc='upper right')
plt.show()

plt.plot(epochsRange, train_acc, label='Training')
plt.plot(epochsRange, val_acc, label='Validation')
plt.title('Accuracy')
plt.legend(loc='upper right')
plt.show()